In [1]:
!pip install pandas
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import copy
import random

In [2]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)
wandb.login()
wandb.init(project ='vanillaRNN')

2024-05-17 05:45:28.320153: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 05:45:28.320294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 05:45:28.425201: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chandraduttamamidi (cs23m021). Use `wandb login --relogin` to force relogin


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
trainCSV = "/kaggle/input/telugu/tel/tel_train.csv"
validCSV = "/kaggle/input/telugu/tel/tel_valid.csv"
testCSV = "/kaggle/input/telugu/tel/tel_test.csv"

In [5]:
trainingData = pd.read_csv(trainCSV, header=None)
validationData = pd.read_csv(validCSV,header = None)
testData = pd.read_csv(testCSV,header= None)


trainingInput = trainingData[0].to_numpy()
trainingOutput = trainingData[1].to_numpy()
#the size of input and output is 4096
validationInput = validationData[0].to_numpy()
validationOutput = validationData[1].to_numpy()


In [6]:
maxLength = 0
maxLengthElement =''
# Loop through elements in validationInput
for element in validationInput:
    maxLength = max(maxLength,len(element))
    if(maxLength == len(element)):
        maxLengthElement=element

maxLength =0 
maxLengthElement =''
# Loop through elements in validationOutput
for element in validationOutput:
    maxLength = max(maxLength,len(element))
    if(maxLength == len(element)):
        maxLengthElement=element    

In [7]:
def preProcessTheData(input, output, validation):
    data = {
        "allCharacters1" : [],
        "charNumMap1" : {},
        "numCharMap1" : {},
        "inputCharToNum": torch.zeros(len(input),30, dtype=torch.int, device=device),
        "inputData" : input,
            
        "allCharacters2" : [],
        "charNumMap2" : {},
        "numCharMap2" : {},
        "outputCharToNum": torch.zeros(len(output),23, dtype=torch.int, device=device),
        "outputData" : output,
        
        "inputLength" : 0,
        "outputLength" : 0
    }
    
    k = 0 

    m1 = data["charNumMap1"]
    m2 = data["charNumMap2"]
    if validation:
        m1 = preProcessedData["charNumMap1"]
        m2 = preProcessedData["charNumMap2"]

    for i in range(0,len(input)):
        input[i] = "{" + input[i] + "}"*(29-len(input[i]))
        charToNum = []
        for char in (input[i]):
            index = 0
            if(char not in data["allCharacters1"]):
                data["allCharacters1"].append(char)
                if validation:
                    index = m1[char]
                else:
                    index = data["allCharacters1"].index(char)
                data["charNumMap1"][char] = index
                data["numCharMap1"][index] = char
            else:
                if validation:
                    index = m1[char]
                else:
                    index = data["allCharacters1"].index(char)
            
            charToNum.append(index)
            
        my_tensor = torch.tensor(charToNum,device = device)
        data["inputCharToNum"][k] = my_tensor
        
        charToNum1 = []
        
        output[i] = "{" + output[i] + "}"*(22-len(output[i]))
        for char in (output[i]):
            index = 0
            if(char not in data["allCharacters2"]):
                data["allCharacters2"].append(char)
                if validation:
                    index = m2[char]
                else:
                    index = data["allCharacters2"].index(char)
                data["charNumMap2"][char] = index
                data["numCharMap2"][index] = char
            else:
                if validation:
                    index = m2[char]
                else:
                    index = data["allCharacters2"].index(char)
                
            charToNum1.append(index)
            
        my_tensor1 = torch.tensor(charToNum1,device = device)
        data["outputCharToNum"][k] = my_tensor1
        
        k+=1
    
    data["inputLength"] = len(data["allCharacters1"])
    data["outputLength"] = len(data["allCharacters2"])
        
    return data

preProcessedData = preProcessTheData(copy.copy(trainingInput),copy.copy(trainingOutput), False)

In [8]:
preprocessedValidationData = preProcessTheData(copy.copy(validationInput),copy.copy(validationOutput), True)


In [9]:
class createDataset(Dataset):
    def __getitem__(self, idx):
        inputData = self.source[idx]
        outputData = self.target[idx]
        return inputData, outputData
    def __len__(self):
        return len(self.source)
    
    def __init__(self, x,y):
        self.source = x
        self.target = y
    

In [10]:
data = preProcessTheData(copy.copy(trainingInput),copy.copy(trainingOutput), False)

def getDataLoader(mode, batchSize):
    if(mode == 'train'):
        dataset = createDataset(data["inputCharToNum"],data['outputCharToNum'])
        return DataLoader(dataset, batch_size=batchSize, shuffle=True)
    else:
        dataset = createDataset(preprocessedValidationData["inputCharToNum"],preprocessedValidationData['outputCharToNum'])
        return  DataLoader(dataset, batch_size=batchSize, shuffle=True)

In [11]:
def evaluateModelPerformance(encoder, decoder, batchSize, temporalFusionParameter):
    
    dataLoader = getDataLoader("validation", batchSize) # Fetch data loader based on the operation mode
    decoder.eval()
    encoder.eval()
    
    
    accumulatedValidationAccuracy = 0
    totalValidationLoss = 0
    
    lossCriterion = nn.NLLLoss()
    
    for batchIndex, (sourceSequence, targetSequence) in enumerate(dataLoader):
        
        initialEncoderState = encoder.getInitialState() 
        
        encoderOutput, currentStateAfterEncoding = encoder(sourceSequence, initialEncoderState)

        cumulativeLoss = 0 

        sequenceLength = targetSequence.shape[1]

        predictedOutputs = []

        randomSelectionFactor = random.random()

        currentDecoderState = currentStateAfterEncoding

        for index in range(sequenceLength):

            if(index == 0):
                inputTensorForDecoder = targetSequence[:, index].view(batchSize, 1) # Reshape to match expected dimensions
            else:
                if randomSelectionFactor < temporalFusionParameter:
                    inputTensorForDecoder = targetSequence[:, index].view(batchSize, 1) # Pass current batch element
                else:
                    inputTensorForDecoder = inputTensorForDecoder.view(batchSize, 1) # Pass previously selected element

            decodedOutput, updatedDecoderState = decoder(inputTensorForDecoder, currentDecoderState)
            topValues, topIndices = decodedOutput.topk(1)  # Retrieve top values and their indices
            inputTensorForDecoder = topIndices.squeeze().detach() # Convert to 1D tensor
            predictedOutputs.append(inputTensorForDecoder) # Append softmax values
                    
            decodedOutput = decodedOutput[:, -1, :] # Reduce size from (batchSize*1*embeddingSize) to (batchSize*embeddingSize)

            targetCharacters = targetSequence[:, index] #(batchSize)
            targetCharacters = targetCharacters.type(dtype=torch.long)

            cumulativeLoss += lossCriterion(decodedOutput, targetCharacters) # Pass softmax values to target characters

        stackedPredictions = torch.stack(predictedOutputs)
        predictionsMatrix = stackedPredictions.transpose(0, 1) # Transpose to match expected format

        accumulatedValidationAccuracy += (predictionsMatrix == targetSequence).all(dim=1).sum().item() # Sum up matching values
        totalValidationLoss += (cumulativeLoss.item()/sequenceLength)

        if(batchIndex % 20 == 0):
            print(f"Batch: {batchIndex}, Loss: {cumulativeLoss.item()/sequenceLength}")
    
    encoder.train()
    decoder.train()
#     print(f"Validation Accuracy: {accumulatedValidationAccuracy/40.96}")
#     print(f"Validation Loss: {totalValidationLoss}")


In [12]:
class Encoder(nn.Module):
    
    def __init__(self, inputDimension, embeddingSize, layerCount, neuronCount, cellType, batchSize):
        super(Encoder, self).__init__()
        self.embeddingLayer = nn.Embedding(inputDimension, embeddingSize)
        self.layerStack = layerCount
        self.neuronCountInHiddenLayer = neuronCount
        self.batchSize = batchSize
        
        if(cellType=='GRU'):
            self.recurrentLayer = nn.GRU(embeddingSize, neuronCount, num_layers=layerCount, batch_first=True)
        elif(cellType=='LSTM'):
            self.recurrentLayer = nn.LSTM(embeddingSize, neuronCount, num_layers=layerCount, batch_first=True)
        else:
            self.recurrentLayer = nn.RNN(embeddingSize, neuronCount, num_layers=layerCount, batch_first=True)
    
    def getInitialState(self):
        return torch.zeros(self.layerStack, self.batchSize, self.neuronCountInHiddenLayer, device=device)
            
    def forward(self, inputData, previousState):
        embeddedInput = self.embeddingLayer(inputData)
        output, stateAfterPassing = self.recurrentLayer(embeddedInput, previousState)
        return output, stateAfterPassing
    
    
class Decoder(nn.Module):
    def __init__(self, outputDimension, embeddingSize, neuronCount, layerCount, cellType, dropoutProbability):
        super(Decoder, self).__init__()
        self.embeddingLayerForOutput = nn.Embedding(outputDimension, embeddingSize)
        
        if(cellType=="GRU"):
            self.recurrentLayerForOutput = nn.GRU(embeddingSize, neuronCount, num_layers=layerCount, batch_first=True)
        elif(cellType=="LSTM"):
            self.recurrentLayerForOutput = nn.LSTM(embeddingSize, neuronCount, num_layers=layerCount, batch_first=True)
        else:
            self.recurrentLayerForOutput = nn.RNN(embeddingSize, neuronCount, num_layers=layerCount, batch_first=True)
            
        self.finalLinearTransformation = nn.Linear(neuronCount, outputDimension)
        self.applySoftMax = nn.LogSoftmax(dim=2)
        self.dropOutLayer = nn.Dropout(dropoutProbability)

    def forward(self, currentInput, previousState):
        embeddedCurrentInput = self.embeddingLayerForOutput(currentInput)
        processedInput = F.relu(embeddedCurrentInput)
        outputFromRecurrent, stateAfterProcessing = self.recurrentLayerForOutput(processedInput, previousState)
        outputFromRecurrent = self.dropOutLayer(outputFromRecurrent)
        finalOutput = self.applySoftMax(self.finalLinearTransformation(outputFromRecurrent))
        return finalOutput, stateAfterProcessing


In [13]:
def num_to_char_converter(source_array, target_array, data):
    num_to_char_map = data['numCharMap2']
    source_string = ''
    target_string = ''
    for source_row, target_row in zip(source_array, target_array):
        source_string = ''
        target_string = ''
        for source_element, target_element in zip(source_row, target_row):
            source_string += num_to_char_map[source_element.item()]
            target_string += num_to_char_map[target_element.item()]
        print(source_string, " ", target_string)

In [14]:
def modelTrainingProcess(embeddingSize, encoderLayerCount, decoderLayerCount, hiddenNeuronsPerLayer, cellType, bidirectionalOption, dropoutRate, numberOfTrainingEpochs, batchSizeForTraining, learningRateValue, chosenOptimizer, temporalFusionThreshold):
   
    dataLoaderInstance = getDataLoader("train", batchSizeForTraining) # Data loader setup for training
    
    encoderInstance = Encoder(data["inputLength"], embeddingSize, encoderLayerCount, hiddenNeuronsPerLayer, cellType, batchSizeForTraining).to(device)
    decoderInstance = Decoder(data["outputLength"], embeddingSize, hiddenNeuronsPerLayer, encoderLayerCount, cellType, dropoutRate).to(device)
    
    if(chosenOptimizer == 'Adam'):
        optimizerForEncoder = optim.Adam(encoderInstance.parameters(), lr=learningRateValue)
        optimizerForDecoder = optim.Adam(decoderInstance.parameters(), lr=learningRateValue)
    else:
        optimizerForEncoder = optim.NAdam(encoderInstance.parameters(), lr=learningRateValue)
        optimizerForDecoder = optim.NAdam(decoderInstance.parameters(), lr=learningRateValue)
    
    lossCalculationMethod = nn.NLLLoss()

    for epochIteration in range(numberOfTrainingEpochs):
    
        accumulatedTrainAccuracy = 0 
        totalTrainLoss = 0 

        for batchIndex, (sourceDataBatch, targetDataBatch) in enumerate(dataLoaderInstance):
                        
            initialEncoderState = encoderInstance.getInitialState() 
            
            if(bidirectionalOption == "Yes"):
                flippedSourceBatch = torch.flip(sourceDataBatch, dims=[1]) # Reverse the batch along rows
                sourceDataBatch = (sourceDataBatch + flippedSourceBatch)//2 # Average reversed and original batches
                
            encodedOutput, currentStateAfterEncoding = encoderInstance(sourceDataBatch, initialEncoderState)
            
            lossAccumulator = 0 
            
            sequenceLength = targetDataBatch.shape[1]

            predictedSequence = []
            
            randomChoiceFactor = random.random()

            currentDecoderState = currentStateAfterEncoding

            for indexPosition in range(sequenceLength):
                
                if(indexPosition == 0):
                    inputTensorForDecoding = targetDataBatch[:, indexPosition].view(batchSizeForTraining, 1) # Reshape to match expected dimensions
                else:
                    if randomChoiceFactor < temporalFusionThreshold:
                        inputTensorForDecoding = targetDataBatch[:, indexPosition].view(batchSizeForTraining, 1) # Pass current batch element
                    else:
                        inputTensorForDecoding = inputTensorForDecoding.view(batchSizeForTraining, 1) # Pass previously selected element

                decodedResult, updatedDecoderState = decoderInstance(inputTensorForDecoding, currentDecoderState)
                topValuedIndices, topIndexPositions = decodedResult.topk(1)  # Get top values and their indices
               
                inputTensorForDecoding = topIndexPositions.squeeze().detach() # Convert to 1D tensor
                predictedSequence.append(inputTensorForDecoding) # Append softmax values
                    
                decodedResult = decodedResult[:, -1, :] # Reduce size from (batchSize*1*embeddingSize) to (batchSize*embeddingSize)

                targetCharacterIndices = targetDataBatch[:, indexPosition] #(batchSize)
                targetCharacterIndices = targetCharacterIndices.type(dtype=torch.long)

                lossAccumulator += lossCalculationMethod(decodedResult, targetCharacterIndices) # Pass softmax values to target characters

            stackedPredictions = torch.stack(predictedSequence)
            predictionsMatrix = stackedPredictions.transpose(0, 1) # Transpose to match expected format

            if(batchIndex == 0 and epochIteration == numberOfTrainingEpochs-1):
                num_to_char_converter(targetDataBatch, predictionsMatrix, data) 

            accumulatedTrainAccuracy += (predictionsMatrix == targetDataBatch).all(dim=1).sum().item() # Sum up matching values
            totalTrainLoss += (lossAccumulator.item()/sequenceLength)
            
            optimizerForEncoder.zero_grad()
            optimizerForDecoder.zero_grad()
            lossAccumulator.backward()
            optimizerForEncoder.step()
            optimizerForDecoder.step()
            
        print(f"Train Accuracy: {accumulatedTrainAccuracy/512}")
        print(f"Train Loss: {totalTrainLoss}")
        evaluateModelPerformance(encoderInstance,decoderInstance,batchSizeForTraining,temporalFusionThreshold)

In [15]:
def main_fun():
    wandb.init(project ='vanillaRNN')
    params = wandb.config
    with wandb.init(project = 'vanillaRNN', name='embedding'+str(params.embSize)+'cellType'+params.cellType+'batchSize'+str(params.batchsize)) as run:
        modelTrainingProcess(params.embSize,params.encoderLayers,params.decoderLayers,params.hiddenLayerNuerons,params.cellType,params.bidirection,params.dropout,params.epochs,params.batchsize,params.learningRate,params.optimizer,params.tf_ratio)
    
sweep_params = {
    'method' : 'bayes',
    'name'   : 'DeepLearningAssignment3',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'validation_accuracy',
    },
    'parameters' : {
        'embSize':{'values':[16,32,64]},
        'encoderLayers':{'values':[1,5,10]},
        'decoderLayers' : {'values' : [1,5,10]},
        'hiddenLayerNuerons'   : {'values' : [64,256,512]},
        'cellType' : {'values' : ['GRU'] } ,
        'bidirection' : {'values' : ['no']},
        'dropout' : {'values' : [0,0.2,0.3]},
        'epochs'  : {'values': [10,20,30]},
        'batchsize' : {'values' : [32,64]},
        'learningRate' : {'values' : [1e-2,1e-3,1e-4]},
        'optimizer':{'values' : ['Adam','Nadam']},
        'tf_ratio' :{'values' : [0.2,0.4,0.5]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'vanillaRNN')
wandb.agent(sweepId,function =main_fun,count = 2)
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 66qndl4s
Sweep URL: https://wandb.ai/cs23m021/vanillaRNN/sweeps/66qndl4s


wandb: Agent Starting Run: r0l68hmn with config:
wandb: 	batchsize: 32
wandb: 	bidirection: no
wandb: 	cellType: GRU
wandb: 	decoderLayers: 1
wandb: 	dropout: 0
wandb: 	embSize: 16
wandb: 	encoderLayers: 1
wandb: 	epochs: 20
wandb: 	hiddenLayerNuerons: 64
wandb: 	learningRate: 0.01
wandb: 	optimizer: Adam
wandb: 	tf_ratio: 0.2


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 286, in check_stop_status
    self._loop_check_status(
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 224, in _loop_check_status
Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    local_handle = request()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/interface/interface.py", line 840, in deliver_stop_status
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    return self._deliver_stop_status(status)
 

...

wandb: Ctrl + C detected. Stopping sweep.


Batch Index: 200, Loss: 1.8888398875360903

Traceback (most recent call last):


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x79141c0ce170>> (for post_run_cell), with arguments args (<ExecutionResult object at 791302910fd0, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 791302913f40, raw_cell="def main_fun():
    wandb.init(project ='vanillaRN.." store_history=True silent=False shell_futures=True cell_id=7fcc390b-34e7-4b04-907c-b612d189218c> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe